In [1]:
!pip install -U transformers accelerate bitsandbytes wikipedia tqdm torch matplotlib

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 10.0 MB 4.9 MB/s            
     |████████████████████████████████| 342 kB 79.0 MB/s            
     |████████████████████████████████| 76.1 MB 215.7 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 78 kB 107.3 MB/s            
     |████████████████████████████████| 766.7 MB 103.4 MB/s            
     |████████████████████████████████| 8.3 MB 149.2 MB/s            
     |████████████████████████████████| 3.0 MB 86.6 MB/s            
     |████████████████████████████████| 64 kB 239.8 MB/s            
     |████████████████████████████████| 468 kB 85.1 MB/s            
     |████████████████████████████████| 780 kB 313.3 MB/s            
     |████████████████████████████████| 19.5 MB 349.9 MB/s            
     |████████████████████████████████| 737 kB 98.1 MB/s            
     |████████████████████████████████| 471 kB 13

In [2]:
!nvidia-smi

Wed Feb 26 20:44:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.03             Driver Version: 550.144.03     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX 4000 Ada Gene...    Off |   00000000:01:00.0 Off |                  Off |
| 30%   41C    P8             11W /  130W |     120MiB /  20475MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
import os
cache_dir = os.getenv("HF_HOME", os.path.expanduser("~/.cache/huggingface"))
print(cache_dir)

/users/eleves-a/2022/zakaria.abboud/.cache/huggingface


In [3]:
# Create a new cache directory -- this is the absolute path to the new cache directory from the root
# of the filesystem. You can change this to any path you want.
# new_cache_dir = "/Data/zakaria.abboud"h
os.chdir("/")
new_cache_dir = "/Data/zakaria.abboud/.cache/huggingface"
if not os.path.exists(new_cache_dir):
    os.makedirs(new_cache_dir, exist_ok=True)

# Set the new cache directory
os.environ["HF_HOME"] = new_cache_dir
print(f"New cache directory: {new_cache_dir}")
print(f"Path from root: {os.path.abspath(new_cache_dir)}")

New cache directory: /Data/zakaria.abboud/.cache/huggingface
Path from root: /Data/zakaria.abboud/.cache/huggingface


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import json
import time
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

/Data/zakaria.abboud/my_venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Define the model name
# model_name = "Qwen/Qwen2.5-7B-Instruct"
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

# Create a BitsAndBytesConfig for 8-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Load the model in 4-bit mode
    bnb_4bit_use_double_quant=True,  # Use double quantization for 4-bit mode
    bnb_4bit_quant_type="nf4",  # Quantization type
    bnb_4bit_compute_dtype=torch.bfloat16,  # Compute dtype
    llm_int8_enable_fp32_cpu_offload=True  # Offload some parts to CPU if needed
)

# Load the model with the quantization config
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto"  # Automatically map model parts to available devices
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

model

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=896, out_features=896, bias=True)
          (k_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (v_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (o_proj): Linear4bit(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear4bit(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (

In [6]:
def summarize_document(document):
    # Tokenize the input document
    prompt = f"<human>: Summarize the following text :\n\n{document}\n\nYou should respond in French, with 5 sentences maximum.\n\n<assistant>:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate summary
    summary_ids = model.generate(**inputs, max_new_tokens=128)

    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    assistant_index = summary.find("<assistant>:")
    if assistant_index != -1:
        summary = summary[assistant_index + len("<assistant>:"):].strip()

    return summary

In [8]:
# test
articles = json.load(open('/users/eleves-a/2022/zakaria.abboud/Desktop/NLP/NLP Projet/summaries/wikipedia_articles.json', 'r', encoding='utf-8'))
test_article = articles[0]["content"]
# print(test_article)

# summarize the test article
summary = summarize_document(test_article)
print(summary)

En français, la chanson "Dans les prisons de Nantes" est une histoire populaire que beaucoup de gens ont entendu et reconnue. Elle a été utilisée pour marquer la fin de certaines prisonniers et pour rappeler la réalité des prisons de Nantes. Elle a été traduite et reproduite par plusieurs artistes, y compris Edith Piaf, Édith Butler, Aristide Bruant, Les Compagnons de la chanson, Nana Mouskouri, Colette Renard, Louise Forestier, Dorothée, le groupe Tri Yann, le ch


In [9]:
# File to save the summarized articles
# output_file = '/users/eleves-a/2022/zakaria.abboud/Desktop/NLP/NLP Projet/summaries/summarized_wikipedia_articles.json'
output_file = '/users/eleves-a/2022/zakaria.abboud/Desktop/NLP/NLP Projet/summaries/bad_summarized_wikipedia_articles.json'

# Function to save results progressively
def save_progress(summarized_articles, output_file):
    if os.path.exists(output_file):
        with open(output_file, 'r', encoding='utf-8') as f:
            existing_data = json.load(f)
    else:
        existing_data = []

    existing_data.extend(summarized_articles)

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(existing_data, f, ensure_ascii=False, indent=4)

# Summarize articles in parallel
summarized_articles = []
with ThreadPoolExecutor(max_workers=4) as executor:  # Adjust max_workers based on your system
    futures = {executor.submit(summarize_document, article['content']): article for article in articles}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Summarizing articles"):
        result = future.result()
        summarized_articles.append({
            'title': futures[future]['title'],
            'content': futures[future]['content'],
            'summary': result
        })

        # Save progress every 10 summaries
        if len(summarized_articles) % 10 == 0:
            save_progress(summarized_articles, output_file)
            summarized_articles = []  # Clear the list after saving

# Save any remaining summaries
if summarized_articles:
    save_progress(summarized_articles, output_file)

print(f"Summarized articles saved to '{output_file}'")

Summarizing articles: 100%|██████████| 4923/4923 [2:19:35<00:00,  1.70s/it]  


Summarized articles saved to '/users/eleves-a/2022/zakaria.abboud/Desktop/NLP/NLP Projet/bad_summarized_wikipedia_articles.json'
